<a href="https://colab.research.google.com/github/amanglarus/image_classification_using-tensorflow/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/tshirt_laptop_data.zip /content/

In [3]:
!unzip tshirt_laptop_data.zip

Archive:  tshirt_laptop_data.zip
   creating: tshirt_laptop_data/
   creating: tshirt_laptop_data/test/
   creating: tshirt_laptop_data/test/laptop/
  inflating: tshirt_laptop_data/test/laptop/n03642806_1017.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1019.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1066.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1104.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1112.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1131.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1177.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1184.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_12.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1202.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1219.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_1221.JPEG  
  inflating: tshirt_laptop_data/test/laptop/n03642806_126.JPE

# Install specific tensorflow version 

In [12]:
!pip install tensorflow-gpu==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=b89f6b5c40d47c77077607402dc8fedc43daa02040599ebfe8d9ab065ffe3ce5
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0

In [20]:
import tensorflow as tf
print(tf.__version__)

2.4.0


# Load data in pipe line to tarin the model

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [7]:
NUM_CLASSES = 2
BATCH_SIZE=32
IMAGE_SIZE = 48
NUM_CHANNEL = 3

IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNEL)

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir='/content/tshirt_laptop_data/train'
val_data_dir='/content/tshirt_laptop_data/test'

# from tensorflow.keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)




train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='sparse')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode='sparse')






Found 2385 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


# Build Model

In [21]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# from tensorflow.keras.applications.mobilenet_v3 import MobileNetV3Small

base_model = tf.keras.applications.MobileNetV3Small(
    input_shape=MODEL_IMAGE_SHAPE,
    alpha=0.2,
    include_top=False,
    weights=None, #  'imagenet'
    #input_tensor=None,
    #pooling=None,
    #classes=1000,
    #classifier_activation='softmax',
    #**kwargs
)

for layer in base_model.layers: #base_model.layers[:-2]
    layer.trainable = True  



# penultimate_layer = model.layers[-2]
# new_top_layer = tf.keras.layers.Dense(1)(penultimate_layer.output)  # create new FC layer and connect it to the rest of the model
# new_model = tf.keras.models.Model(model.input, new_top_layer)



# Use a Sequential model to add a trainable classifier on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3small (Functional (None, 2, 2, 1024)        231608    
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 297,338
Trainable params: 294,058
Non-trainable params: 3,280
____________________________________________

# Train the model

In [22]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

model.compile(loss='SparseCategoricalCrossentropy',
                   optimizer=Adam(learning_rate=0.0015),
                   metrics=['sparse_categorical_accuracy'])

epochs=1

batch_size = 32

history = model.fit(train_generator,
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #  callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                         )

75/75 [==============================] - 30s 312ms/step - loss: 0.7054 - sparse_categorical_accuracy: 0.5513 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.5000


# Convert from tensorflow to tensorflow lite

In [23]:
datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
generator=datagen.flow_from_directory(val_data_dir,
                                      target_size=IMAGE_SHAPE,
                                      batch_size=BATCH_SIZE,
                                      class_mode='sparse')
def representative_data_gen():
  i = 0
  for image_batch, labels_batch in generator:
    i = i+1
    if i > 20:
      break;
    yield [image_batch]

Found 200 images belonging to 2 classes.


In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

print('\nSetting the optimization flags..')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print('\nConverting..')
tflite_model = converter.convert()

open("person_detect_model_data.tflite", "wb").write(tflite_model)




Setting the optimization flags..

Converting..


501480

# Convert from tensorflow to c array

In [25]:
!xxd -i ./person_detect_model_data.tflite > person_detect_model_data.cpp

In [48]:
#!cp /content/person_detect_model_data.cpp /content/person_detect_model_data_copy.cpp

In [26]:

search_text1 = "unsigned char __person_detect_model_data_tflite[]"


replace_text1 = str('#include "person_detect_model_data.h" \n\
#ifdef __has_attribute \n\
#define HAVE_ATTRIBUTE(x) __has_attribute(x) \n\
#else \n\
#define HAVE_ATTRIBUTE(x) 0 \n\
#endif \n\
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__)) \n\
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4))) \n\
#else \n\
#define DATA_ALIGN_ATTRIBUTE \n\
#endif \n\
const unsigned char g_person_detect_model_data[] DATA_ALIGN_ATTRIBUTE')


search_text2 = "unsigned int __person_detect_model_data_tflite_len"

replace_text2 = "const int g_person_detect_model_data_len"

with open(r'/content/person_detect_model_data.cpp', 'r') as file:

    data = file.read()

    data = data.replace(search_text1, replace_text1)
    data = data.replace(search_text2, replace_text2)
  # data = data.replace(search_text2, replace_text2)

with open(r'/content/person_detect_model_data.cpp', 'w') as file:
    file.write(data)
    file.close()



print("Text replaced")


Text replaced
